In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))


def po_linear_model(graph, alpha=1, beta=1, sigma=0.1, gamma=2):    
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i]  

def po_multiplicative_model(graph, alpha=1, sigma=0.1, delta=1, gamma=2): 
    for i in graph.nodes:
        graph.nodes[i]["y"] = ( (alpha + sigma * np.random.normal()) * graph.degree[i]/avg_deg )  *  (1 + delta * graph.nodes[i]["z"] + gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]]) / len(graph[i]) )




DGL backend not selected or invalid.  Assuming PyTorch for now.
/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [3]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [4]:
dgl_G = dgl.from_networkx(g)

In [9]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [6]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)

        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [ ]:
# cluster-level cr

GATE_hat_list = np.zeros((repeat_num, len(ramps)))

     
for seed in tqdm(range(repeat_num)):    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_cluster,))   
    
    for num_step in range(len(ramps)): 
        print(f"Seed: {seed}")        
        torch.manual_seed(2)
        p_list = [0, ramps[num_step]]                
        g_feat_list = []
        y_list = []
        
        for p in p_list:            
            tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]
            nx.set_node_attributes(g, 0, "z")

            if len(tr_clusters) > 0:
                tr_units = reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters])                  
                nx.set_node_attributes(g, {unit:1 for unit in tr_units}, "z")
            
            po_linear_model(g, gamma = 1)            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()    
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
            
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
                    
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    # torch.save(GATE_hat_list, "Result/2step_CR_cluster.pkl")
    torch.save(GATE_hat_list, "Result/incre_2step_CR_cluster.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.00, Loss: 8.1246
Epoch 0, Treat Prop: 0.02, Loss: 2.3955
Epoch 50, Treat Prop: 0.00, Loss: 0.1727
Epoch 50, Treat Prop: 0.02, Loss: 0.1859
Epoch 100, Treat Prop: 0.00, Loss: 0.0243
Epoch 100, Treat Prop: 0.02, Loss: 0.0421
Epoch 150, Treat Prop: 0.00, Loss: 0.0138
Epoch 150, Treat Prop: 0.02, Loss: 0.0253
Epoch 200, Treat Prop: 0.00, Loss: 0.0126
Epoch 200, Treat Prop: 0.02, Loss: 0.0202
Epoch 250, Treat Prop: 0.00, Loss: 0.0120
Epoch 250, Treat Prop: 0.02, Loss: 0.0166
Epoch 300, Treat Prop: 0.00, Loss: 0.0118
Epoch 300, Treat Prop: 0.02, Loss: 0.0150
Epoch 350, Treat Prop: 0.00, Loss: 0.0117
Epoch 350, Treat Prop: 0.02, Loss: 0.0141
0.013500920496881008
Seed: 0
Epoch 0, Treat Prop: 0.00, Loss: 8.1190
Epoch 0, Treat Prop: 0.05, Loss: 2.4119
Epoch 50, Treat Prop: 0.00, Loss: 0.1612
Epoch 50, Treat Prop: 0.05, Loss: 0.2198
Epoch 100, Treat Prop: 0.00, Loss: 0.0266
Epoch 100, Treat Prop: 0.05, Loss: 0.0713
Epoch 150, Treat Prop: 0.00, Loss: 0.0160
Epoch 150, Treat 

  0%|          | 1/1000 [00:21<6:00:11, 21.63s/it]

...

0.04724742844700813
Seed: 378
Epoch 0, Treat Prop: 0.00, Loss: 8.1136
Epoch 0, Treat Prop: 0.02, Loss: 2.4046
Epoch 50, Treat Prop: 0.00, Loss: 0.1579
Epoch 50, Treat Prop: 0.02, Loss: 0.2239
Epoch 100, Treat Prop: 0.00, Loss: 0.0259
Epoch 100, Treat Prop: 0.02, Loss: 0.0722
Epoch 150, Treat Prop: 0.00, Loss: 0.0157
Epoch 150, Treat Prop: 0.02, Loss: 0.0435
Epoch 200, Treat Prop: 0.00, Loss: 0.0130
Epoch 200, Treat Prop: 0.02, Loss: 0.0266
Epoch 250, Treat Prop: 0.00, Loss: 0.0111
Epoch 250, Treat Prop: 0.02, Loss: 0.0189
Epoch 300, Treat Prop: 0.00, Loss: 0.0111
Epoch 300, Treat Prop: 0.02, Loss: 0.0152
Epoch 350, Treat Prop: 0.00, Loss: 0.0112
Epoch 350, Treat Prop: 0.02, Loss: 0.0150
0.016106098890304565
Seed: 378
Epoch 0, Treat Prop: 0.00, Loss: 8.1196
Epoch 0, Treat Prop: 0.05, Loss: 2.6899
Epoch 50, Treat Prop: 0.00, Loss: 0.1995
Epoch 50, Treat Prop: 0.05, Loss: 0.4225
Epoch 100, Treat Prop: 0.00, Loss: 0.0354
Epoch 100, Treat Prop: 0.05, Loss: 0.1914
Epoch 150, Treat Prop: 0.00

 38%|███▊      | 379/1000 [2:14:04<3:36:22, 20.91s/it]

0.043943025171756744
Seed: 379
Epoch 0, Treat Prop: 0.00, Loss: 8.1243
Epoch 0, Treat Prop: 0.02, Loss: 2.3588
Epoch 50, Treat Prop: 0.00, Loss: 0.1599
Epoch 50, Treat Prop: 0.02, Loss: 0.2140
Epoch 100, Treat Prop: 0.00, Loss: 0.0311
Epoch 100, Treat Prop: 0.02, Loss: 0.0733
Epoch 150, Treat Prop: 0.00, Loss: 0.0164
Epoch 150, Treat Prop: 0.02, Loss: 0.0383
Epoch 200, Treat Prop: 0.00, Loss: 0.0136
Epoch 200, Treat Prop: 0.02, Loss: 0.0244
Epoch 250, Treat Prop: 0.00, Loss: 0.0121
Epoch 250, Treat Prop: 0.02, Loss: 0.0171
Epoch 300, Treat Prop: 0.00, Loss: 0.0112
Epoch 300, Treat Prop: 0.02, Loss: 0.0141
Epoch 350, Treat Prop: 0.00, Loss: 0.0112
Epoch 350, Treat Prop: 0.02, Loss: 0.0138
0.013545313850045204
Seed: 379
Epoch 0, Treat Prop: 0.00, Loss: 8.1180
Epoch 0, Treat Prop: 0.05, Loss: 2.3844
Epoch 50, Treat Prop: 0.00, Loss: 0.1566
Epoch 50, Treat Prop: 0.05, Loss: 0.2606
Epoch 100, Treat Prop: 0.00, Loss: 0.0294
Epoch 100, Treat Prop: 0.05, Loss: 0.0942
Epoch 150, Treat Prop: 0.0

 38%|███▊      | 380/1000 [2:14:26<3:37:14, 21.02s/it]

0.04302356764674187
Seed: 380
Epoch 0, Treat Prop: 0.00, Loss: 8.1149
Epoch 0, Treat Prop: 0.02, Loss: 2.4324
Epoch 50, Treat Prop: 0.00, Loss: 0.1780
Epoch 50, Treat Prop: 0.02, Loss: 0.2111
Epoch 100, Treat Prop: 0.00, Loss: 0.0240
Epoch 100, Treat Prop: 0.02, Loss: 0.0602
Epoch 150, Treat Prop: 0.00, Loss: 0.0147
Epoch 150, Treat Prop: 0.02, Loss: 0.0370
Epoch 200, Treat Prop: 0.00, Loss: 0.0136
Epoch 200, Treat Prop: 0.02, Loss: 0.0216
Epoch 250, Treat Prop: 0.00, Loss: 0.0112
Epoch 250, Treat Prop: 0.02, Loss: 0.0156
Epoch 300, Treat Prop: 0.00, Loss: 0.0111
Epoch 300, Treat Prop: 0.02, Loss: 0.0147
Epoch 350, Treat Prop: 0.00, Loss: 0.0116
Epoch 350, Treat Prop: 0.02, Loss: 0.0148
0.031378135085105896
Seed: 380
